### To run the cbgt network, see first the technical_testbench to install all the required packages and start the server 

In [ ]:
#Auto reloading 
#%load_ext autoreload
#%autoreload 2
#%reload_ext autoreload 

In [ ]:
#Removing indentation issues:
#!autopep8 --in-place --aggressive --aggressive init_params.py
#!autopep8 --in-place --aggressive --aggressive popconstruct.py
#!autopep8 --in-place --aggressive --aggressive generateepochs.py
#!autopep8 --in-place --aggressive --aggressive qvalues.py

In [1]:
!python setup.py build_ext --inplace

running build_ext
copying build/lib.linux-x86_64-3.6/agent_timestep.cpython-36m-x86_64-linux-gnu.so -> 


In [2]:
#Importing scripts:
import cbgt as cbgt
from frontendhelpers import * 
from tracetype import *
import init_params as par 
import popconstruct as popconstruct
import qvalues as qval
import generateepochs as gen
from agentmatrixinit import *
#from general_init_codeblocks import *
from agent_timestep import timestep_mutator, multitimestep_mutator
import pdb
import pipeline_creation as pl_creat
#import mega_loop as ml


/home/bahuguna/anaconda3/envs/cbgt_env/lib/python3.6/site-packages/ray/autoscaler/_private/cli_logger.py:61: FutureWarning: Not all Ray CLI dependencies were found. In Ray 1.4+, the Ray CLI, autoscaler, and dashboard will only be usable via `pip install 'ray[default]'`. Please update your install command.
  "update your install command.", FutureWarning)
2021-09-13 10:51:33,904	INFO worker.py:727 -- Connecting to existing Ray cluster at address: 172.25.0.163:6379


# Create pipelines

### Network pipeline

In [3]:
# Create the whole pipeline, this pipeline creates three sub-pipelines: reward(rsg), q value(q_val), network(pl)
pl = pl_creat.create_main_pipeline()

# Timestep loop

In [4]:
def mega_loop(slf):
    slf.AMPA_con,slf.AMPA_eff = CreateSynapses(slf.popdata,slf.connectivity_AMPA,slf.meaneff_AMPA,slf.plastic_AMPA)
    slf.GABA_con,slf.GABA_eff = CreateSynapses(slf.popdata,slf.connectivity_GABA,slf.meaneff_GABA,slf.plastic_GABA)
    slf.NMDA_con,slf.NMDA_eff = CreateSynapses(slf.popdata,slf.connectivity_NMDA,slf.meaneff_NMDA,slf.plastic_NMDA)

    popdata = slf.popdata
    actionchannels = slf.actionchannels
    agent = initializeAgent(popdata)
    slf.agent = agent

    agent.AMPA_con,agent.AMPA_eff = slf.AMPA_con,slf.AMPA_eff
    agent.GABA_con,agent.GABA_eff = slf.GABA_con,slf.GABA_eff
    agent.NMDA_con,agent.NMDA_eff = slf.NMDA_con,slf.NMDA_eff
    agent.LastConductanceNMDA = CreateAuxiliarySynapseData(popdata,slf.connectivity_NMDA)

    agent.FRs = agent.rollingbuffer.mean(1) / untrace(list(popdata['N'])) / agent.dt * 1000

    multitimestep_mutator(agent,popdata,1000)

    agent.phase = 0
    agent.phasetimer = 0
    agent.motor_queued = None
    agent.dpmn_queued = None
    agent.gain = np.ones(len(actionchannels))
    agent.extstim = np.zeros(len(actionchannels))
    agent.ramping_extstim = np.zeros(len(actionchannels))
    agent.in_popids = np.where(popdata['name'] == 'LIP')[0]
    agent.out_popids = np.where(popdata['name'] == 'Th')[0]

    presented_stimulus = 1
    slf.chosen_action = None

    while slf.trial_num < slf.n_trials:
        agent.extstim = agent.gain * presented_stimulus * 4.0  # TODO: make 3.0 a param
        agent.ramping_extstim = agent.ramping_extstim * 0.9 + agent.extstim * 0.1
        for action_idx in range(len(actionchannels)):
            popid = agent.in_popids[action_idx]
            agent.FreqExt_AMPA[popid] = np.ones(len(agent.FreqExt_AMPA[popid])) * agent.ramping_extstim[action_idx]
        multitimestep_mutator(agent,popdata,5)
        agent.phasetimer += 1 # 1 ms = 5 * dt
        #agent.FRs = np.stack((agent.FRs,agent.rollingbuffer.mean(1) / untrace(list(popdata['N'])) / agent.dt * 1000))

        if agent.phase == 0:
            gateFRs = agent.rollingbuffer[agent.out_popids].mean(1) / untrace(list(popdata['N'][agent.out_popids])) / agent.dt * 1000
            thresholds_crossed = np.where(gateFRs > 30)[0]
            if len(thresholds_crossed) > 0 or agent.phasetimer > 1000:
                agent.phase = 1
                agent.phasetimer = 0
                agent.gain = np.zeros(len(actionchannels))
                if len(thresholds_crossed) > 0:
                    agent.motor_queued = thresholds_crossed[0]
                    agent.gain[agent.motor_queued] = 0.5
                else:
                    agent.motor_queued = -1

        if agent.phase == 1:
            if agent.phasetimer > 100:
                agent.phase = 2
                agent.phasetimer = 0
                agent.gain = np.zeros(len(actionchannels))
                print(actionchannels)
                slf.chosen_action = untrace(actionchannels.iloc[agent.motor_queued,0])
                print("chosen_action",slf.chosen_action)
                agent.motor_queued = None

        if agent.phase == 2:
            if agent.phasetimer > 100:
                slf.dpmndefaults['dpmn_DAp'] = 0
                slf.trial_num += 1
                agent.phase = 0
                agent.phasetimer = 0


        # environment

        if slf.chosen_action is not None:
            #slf.reward_val = qval.get_reward_value(slf.t1_epochs,slf.t2_epochs,slf.chosen_action,slf.trial_num)
            slf.reward_val = qval.get_reward_value(slf.t_epochs,slf.chosen_action,slf.trial_num)
            slf.Q_support_params = qval.helper_update_Q_support_params(slf.Q_support_params,slf.reward_val,slf.chosen_action)
            (slf.Q_df, slf.Q_support_params, slf.dpmndefaults) = qval.helper_update_Q_df(slf.Q_df,slf.Q_support_params,slf.dpmndefaults,slf.trial_num)
            slf.chosen_action = None

In [5]:
pl.add(mega_loop)

# Running the pipeline

### 1. Default values

In [6]:
#See init_params.py / generateepochs.py / popconstruct.py / qvalues.py
#to know the parameters contained in each of the following variables 
environment = {
    'params': None, #neuron parameters (init_params.py)
    'pops': None,   #population parameters (init_params.py)
    'receps' : None, #receptor parameters (init_params.py)
    'base' : None, #baseline stimulation parameters (init_params.py)
    'dpmns' : None, #dopamine related parameters (init_params.py)
    'd1' : None, #D1-MSNs population related parameters (init_params.py)
    'd2' : None, #D2-MSNs population related parameters (init_params.py)
    'channels' : pd.DataFrame([["left"],["right"]], columns=['action']), #action channels related parameters (init_params.py)
    #'actionchannels' : pd.DataFrame([[1],[2]], columns=['action']), #labels for the actions (init_params.py)
    #'actionchannels' : pd.DataFrame([["left"],["right"]], columns=['action']), #labels for the actions (init_params.py)
    'newpathways' : None, #connectivity parameters (popconstruct.py)
    'Q_support_params': None, #initialization of Q-values update (qvalues.py) 
    'Q_df':None, #initialized Q-values df (qvalues.py)  
    'n_trials': 5, #number of trials (generateepochs.py)
    'volatility': 2, #frequency of changepoints (generateepochs.py)
    'conflict': 0.7, #probability of the preferred choice (generateepochs.py)
    'reward_mu': 1, #mean for the magnitude of the reward (generateepochs.py)
    'reward_std': 0.1, #std for the magnitude of the reward (generateepochs.py)
}
results = cbgt.ExecutionManager(cores=7).run(pl,environment)

(raylet) /home/bahuguna/anaconda3/envs/cbgt_env/lib/python3.6/site-packages/ray/autoscaler/_private/cli_logger.py:61: FutureWarning: Not all Ray CLI dependencies were found. In Ray 1.4+, the Ray CLI, autoscaler, and dashboard will only be usable via `pip install 'ray[default]'`. Please update your install command.
(raylet)   "update your install command.", FutureWarning)


(pid=229649)   action
(pid=229649) 0   left
(pid=229649) 1  right
(pid=229649) define_epochs
(pid=229649)   action
(pid=229649) 0   left
(pid=229649) 1  right
(pid=229649) reward 
(pid=229649)        left     right
(pid=229649) 0  0.000000  0.902428
(pid=229649) 1  0.955901  0.000000
(pid=229649) 2  0.000000  1.182346
(pid=229649) 3  1.039562  0.000000
(pid=229649) 4  0.986540  0.000000
(pid=229649)   action
(pid=229649) 0   left
(pid=229649) 1  right
(pid=229649) 2 4 0.161668`
(pid=229649) 3 5 0.161668`
(pid=229649) 10 6 1.0`
(pid=229649) 10 8 1.0`
(pid=229649) 10 10 0.4335`
(pid=229649) 10 11 0.4335`
(pid=229649) 10 12 1.0`
(pid=229649) 10 13 1.0`
(pid=229649) 10 14 1.0`
(pid=229649) 10 15 0.241667`
(pid=229649) 11 7 1.0`
(pid=229649) 11 9 1.0`
(pid=229649) 11 10 0.4335`
(pid=229649) 11 11 0.4335`
(pid=229649) 11 12 1.0`
(pid=229649) 11 13 1.0`
(pid=229649) 11 14 1.0`
(pid=229649) 11 15 0.241667`
(pid=229649) 12 6 1.0`
(pid=229649) 12 8 1.0`
(pid=229649) 12 14 0.8334`
(pid=229649) 13

In [ ]:
results.keys()

In [ ]:
results['actionchannels']

In [ ]:
results['Q_df']

In [ ]:
results['chosen_action']

### 2. Modifying default values 

### 2.1 Modifying values 

In [ ]:
environment = {
    'params': pd.DataFrame([[8, 30]], columns=['N', 'C']),
    'pops' : {'FSI': {'Taum': [60]}}, 
    'receps' : pd.DataFrame([[100, 100]], columns=['Tau_AMPA', 'Tau_GABA']), 
    'base' : {'FSI': {'FreqExt_AMPA': [100]}}, 
    'dpmns' : pd.DataFrame([[400, 100]], columns=['dpmn_wmax', 'dpmn_tauDOP']),
    'd1' : pd.DataFrame([[3, 50]], columns=['dpmn_type', 'dpmn_c']),
    'd2' : pd.DataFrame([[3, 50]], columns=['dpmn_type', 'dpmn_c']),
    'channels' : None,
    'actionchannels' : pd.DataFrame([[1],[2]], columns=['action']), 
    'newpathways' : pd.DataFrame([['LIP', 'D1STR', 'AMPA', 'syn', 700, 0.027]], columns=['src', 'dest', 'receptor', 'type', 'con', 'eff']), 
    'Q_support_params': None,
    'Q_df':None,
    'n_trials': 5,
    'volatility': 2,
    'conflict': 0.7,
    'reward_mu': 1,
    'reward_std': 0.1,
}
results = cbgt.ExecutionManager(cores=7).run(pl,environment) 

In [ ]:
results['popdata']

### 2.2 Modifying a subset of values

In [ ]:
environment = {
    'params': pd.DataFrame([[4, 1]], columns=['N', 'C']), 
    'pops': None,
    'receps' : None, 
    'base' : None,
    'dpmns' : None,
    'd1' : None,
    'd2' : None,
    'channels' : None,
    'actionchannels' : pd.DataFrame([[1],[2]], columns=['action']), 
    'newpathways' : None, 
    'Q_support_params': None,
    'Q_df':None,
    'n_trials': 5,
    'volatility': 2,
    'conflict': 0.7,
    'reward_mu': 1,
    'reward_std': 0.1,
}

results = cbgt.ExecutionManager(cores=7).run(pl,environment) 

In [ ]:
results['popdata']

### 3. Back to default values 

In [ ]:
#See init_params.py / generateepochs.py / popconstruct.py / qvalues.py
#to know the parameters contained in each of the following variables 
environment = {
    'params': None, #neuron parameters (init_params.py)
    'pops': None,   #population parameters (init_params.py)
    'receps' : None, #receptor parameters (init_params.py)
    'base' : None, #baseline stimulation parameters (init_params.py)
    'dpmns' : None, #dopamine related parameters (init_params.py)
    'd1' : None, #D1-MSNs population related parameters (init_params.py)
    'd2' : None, #D2-MSNs population related parameters (init_params.py)
    'channels' : None, #action channels related parameters (init_params.py)
    'actionchannels' : pd.DataFrame([[1],[2]], columns=['action']), #labels for the actions (init_params.py)
    'newpathways' : None, #connectivity parameters (popconstruct.py)
    'Q_support_params': None, #initialization of Q-values update (qvalues.py) 
    'Q_df':None, #initialized Q-values df (qvalues.py)  
    'n_trials': 5, #number of trials (generateepochs.py)
    'volatility': 2, #frequency of changepoints (generateepochs.py)
    'conflict': 0.7, #probability of the preferred choice (generateepochs.py)
    'reward_mu': 1, #mean for the magnitude of the reward (generateepochs.py)
    'reward_std': 0.1, #std for the magnitude of the reward (generateepochs.py)
}
results = cbgt.ExecutionManager(cores=7).run(pl,environment) 

In [ ]:
results['popdata']

In [ ]:
#?

In [ ]:
dev = cbgt.Pipeline()

In [ ]:
dev.add(codeblock_agent_phase0)

In [ ]:
output = cbgt.ExecutionManager(cores=7).run(dev,results) 